In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

In [2]:
times=pd.read_csv('serial\\times.csv')

In [3]:
times.drop_duplicates(subset=['scheme','function','npoints','CFL'], keep='last', inplace=True)
cols=['L1','L2','LINF','L1_t=5','L2_t=5','LINF_t=5']
times.rename(columns={'L1_m': 'L1_t=5', 'L2_m': 'L2_t=5', 'LINF_m': 'LINF_t=5'}, inplace=True)
times[cols] = times[cols].apply(pd.to_numeric, errors='coerce')

In [4]:
times

,scheme,function,npoints,CFL,do-time,itavgtime,extime,writetime,L1,L2,LINF,L1_t=5,L2_t=5,LINF_t=5
0,upw,sgn,200,0.5,0.000230,0.000003,0.003867,0.003383,3.464364,1.008392,4.825637e-01,2.450924,0.850383,4.801139e-01
1,upw,sgn,800,0.5,0.001114,0.000004,0.008556,0.007171,6.905345,1.421844,4.856202e-01,4.878176,1.194285,4.735800e-01
2,upw,sgn,1600,0.5,0.001142,0.000002,0.010843,0.009533,9.768839,1.691330,4.898264e-01,6.904330,1.421437,4.813020e-01
3,upw,sgn,6400,0.5,0.010382,0.000004,0.048335,0.037567,19.542494,2.392394,4.949111e-01,13.816988,2.011474,4.906449e-01
4,cnt,sgn,200,0.5,0.000281,0.000004,0.004633,0.004044,5483.547030,1050.347160,3.529930e+02,95.436781,21.041718,8.175327e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,lax,exp,6400,1.0,0.289899,0.000241,0.441056,0.148721,0.052921,0.003343,3.855140e-04,0.026470,0.001673,1.928653e-04
100,mcc,exp,200,0.1,0.008110,0.000022,0.013553,0.004792,2.422581,0.572781,2.466522e-01,1.468801,0.417376,2.076454e-01
101,mcc,exp,6400,0.1,6.205105,0.000517,6.339898,0.131850,0.117108,0.007500,7.561802e-04,0.058557,0.003750,3.777739e-04
102,mcc,exp,200,1.0,0.001374,0.000034,0.006886,0.004883,0.119011,0.041013,2.342780e-02,0.056348,0.020863,1.137867e-02


In [5]:
schemes = times.scheme.drop_duplicates().values
schemesdict=dict(zip(schemes,['Upwind','FTCS','Lax-Wendroff','Third Order Upwind','Lax-Friedrichs','MacCormack']))

In [6]:
functions=['sgn','exp']
functionsdict=dict(zip(functions,['Step','Exp Wave']))

In [12]:
number=0
plt.figure(figsize=(10,10))
for k in functions:
    for i in schemes:
        temp = times[(times.scheme == i) & (times.CFL == 0.5) & (times.function == k)]
        temp = temp.copy()
        for j in cols:
            temp[j]=temp[j]/temp['npoints']
            if '=' in j:
                plt.plot('npoints', j, data=temp,linestyle='dashed')
            else:
                plt.plot('npoints', j, data=temp)
        plt.title('Error norms ' + schemesdict[i],fontsize='20')
        plt.xlabel('Number of points',fontsize='18')
        plt.tick_params(axis='both',labelsize=20)
        plt.legend(prop={'size': 20})
        plt.savefig('norms_' + i + '_' + k + '_' + 'CFL05.png',dpi=200)
        #number=number + 1
        #if number == 1:
        #    break
        plt.clf()
    #break

<Figure size 720x720 with 0 Axes>

In [16]:
times[(times.CFL == 0.5) & (times.npoints == 6400) & (times.scheme != 'cnt')]

,scheme,function,npoints,CFL,do-time,itavgtime,extime,writetime,L1,L2,LINF,L1_t=5,L2_t=5,LINF_t=5
3,upw,sgn,6400,0.5,0.010382,0.000004,0.048335,0.037567,19.542494,2.392394,0.494911,13.816988,2.011474,0.490645
15,lxw,sgn,6400,0.5,0.022845,0.000010,0.060984,0.037732,10.790669,1.661578,0.646046,8.149120,1.453564,0.634701
23,upw,exp,6400,0.5,0.010369,0.000004,0.049922,0.038892,5.893548,0.364468,0.041179,3.074638,0.192118,0.021914
35,lxw,exp,6400,0.5,0.022623,0.000009,0.062540,0.039243,0.088725,0.005682,0.000573,0.044361,0.002841,0.000286
75,lax,sgn,6400,0.5,0.548265,0.000228,0.695403,0.144932,33.847705,3.148342,0.490795,23.930224,2.647005,0.487673
79,mcc,sgn,6400,0.5,1.226878,0.000511,1.359344,0.130621,10.790669,1.661578,0.646046,8.149120,1.453564,0.634701
83,lax,exp,6400,0.5,0.580173,0.000242,0.723589,0.140676,15.252060,0.911072,0.100034,8.494566,0.520330,0.058297
87,mcc,exp,6400,0.5,1.299949,0.000542,1.501972,0.198293,0.088725,0.005682,0.000573,0.044361,0.002841,0.000286
